# K Nearest Neighbour Search 

A k-nearest neighbor (kNN) search finds the k nearest vectors to a query vector, as measured by a similarity metric. 

We will use the kNN algorithm to find some nearest neighbors and then compute the recall of the neighborhood we found. 

In [44]:
# Import libraries
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt
import re
import string
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle
from sklearn.preprocessing import StandardScaler
import math
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import warnings
warnings.filterwarnings("ignore")
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/david/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

We will be using Amazon Reviews Dataset. This dataset consists of a few million Amazon customer reviews (input text) and star ratings (output labels). Additional information found here: https://s3.amazonaws.com/amazon-reviews-pds/readme.html

In [45]:
# Read the dataset
data = pd.read_csv('Reviews.csv')

In [46]:
# print out the shape of the dataframe
data.shape

(20000, 10)

In [47]:
data.head(10)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


Text preprocessing 

We will be removing stop-words, any punctuations or limited set of special characters like , or . or # etc. Then, we go on to snowball stemming the word and convert it to lowercase.

In [48]:
# Set of stopwords
stop = set(stopwords.words('english'))

# Initialising the snowball stemmer
sno = nltk.stem.SnowballStemmer('english') 

# Function to clean the word of any html-tags
def cleanhtml(sentence): 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext

# Function to clean the word of any punctuation or special characters
def cleanpunc(sentence): 
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

i=0
str1=' '
final_string=[]

# Store words from +ve reviews 
all_positive_words=[]

# Store words from -ve reviews
all_negative_words=[] 
s=''

final_string=[]
all_positive_words=[] 
all_negative_words=[] 
s=''
for sent in data['Text'].values:
    filtered_sentence=[]

    # Remove HTMl tags
    sent=cleanhtml(sent) 
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                else:
                    continue
            else:
                continue 
    str1 = b" ".join(filtered_sentence) #final string of cleaned words    
    final_string.append(str1)
    i+=1

Generate bag of words vector matrix for reviews.

In [49]:
count_vect = CountVectorizer() 
final_bow_count = count_vect.fit_transform(final_string)

In [50]:
final_bow_np = StandardScaler(with_mean=False).fit_transform(final_bow_count )

Split the data into Train and Test into a 9:1 split.

In [51]:
X = final_bow_np

X_train =  final_bow_np[:math.ceil(len(data)*.9)] 
X_test = final_bow_np[math.ceil(len(data)*.9):]

Find the 50 nearest neighbors for each sample in the test set.

In [52]:
%%time
k = 50
nbrs = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='cosine').fit(X_train)
distances, neighbors = nbrs.kneighbors(X_test, k)

CPU times: user 1.23 s, sys: 760 ms, total: 1.99 s
Wall time: 2 s


In [53]:
distances

array([[6.18464162e-01, 7.16116166e-01, 7.55184269e-01, ...,
        9.39110042e-01, 9.39593503e-01, 9.39595312e-01],
       [6.84371145e-01, 7.00441545e-01, 7.02512885e-01, ...,
        8.69142303e-01, 8.70446927e-01, 8.70674543e-01],
       [5.34895648e-01, 6.37843018e-01, 6.56247624e-01, ...,
        9.12725473e-01, 9.13171758e-01, 9.13337868e-01],
       ...,
       [0.00000000e+00, 6.27357430e-01, 6.80607607e-01, ...,
        9.17175649e-01, 9.18609906e-01, 9.18658569e-01],
       [0.00000000e+00, 4.72554556e-01, 6.08074531e-01, ...,
        9.28125513e-01, 9.28183424e-01, 9.28417540e-01],
       [2.22044605e-16, 5.65968666e-01, 5.89778151e-01, ...,
        8.88874262e-01, 8.88950398e-01, 8.89400426e-01]])

In [54]:
neighbors

array([[12186, 13757, 13091, ..., 13728, 12408,  7513],
       [ 9565,  7384, 10521, ...,  9948, 12740, 10799],
       [17992, 17338, 17998, ..., 15123,   352,   807],
       ...,
       [11088, 12352, 11107, ...,  1771,  4423,  3060],
       [11089, 17989,  7498, ...,  9398,  7994,  1629],
       [11090, 11543, 11687, ...,  1527, 15503,  7154]])

Compute the average recall of the found neighbors. For each sample, the recall is computed as the number of found neighbors divided by the number of neighbors to be found.

In [55]:
def recall(pred, gt):
    """Compute the recall of the pred neighborhood vs. the gt neighborhood"""
    if pred.shape[0] == 1:
        return len(set(pred).intersection(set(gt)))/len(gt)
    assert pred.shape[0] == gt.shape[0]
    return np.array([len(set(pred[i]).intersection(set(gt[i])))/len(gt[i]) for i in range(len(pred))]).mean()

In [56]:
# recall for a random assignment of neighbors
recall(np.random.randint(len(neighbors), size=(len(neighbors), k)), neighbors)

0.0028399999999999996

In [57]:
# recall if the last 3 neighbors were missed for each sample
recall(neighbors[:, range(k-3)], neighbors)

0.9399999999999995

In [58]:
# recall if you found all the correct neighbors
recall(neighbors, neighbors)

1.0

# References

1. https://scikit-learn.org/stable/modules/neighbors.html
3. https://medium.com/analytics-vidhya/k-nearest-neighbor-algorithm-with-amazon-food-reviews-analysis-14d83a4cadea 
4. https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html



# Task

1. Execute Approximate Nearest Neighbor search using TWO of the tools of your choice from the ANN benchmarks. https://ann-benchmarks.com/. Please visit http://github.com/erikbern/ann-benchmarks/ to get an overview of evaluated algorithms and their performance.

2. Train your ANN models for k=50 using the Train-Test split above and tune their parameters to achieve a recall of at least 80%.

3. For each method, display the recall as well as the overall time (training and search) for the search process.